<a href="https://colab.research.google.com/github/kaarthik-balakrishnan/MapMySections/blob/main/MapMySections_Starter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧠 MapMySections Starter Notebook
This notebook sets up your environment, loads sample brain section data, and runs a basic image registration pipeline using `SimpleITK`.

**Steps included:**
- Install dependencies
- Load sample brain section
- Register to a synthetic reference atlas
- Visualize results
- Sync to GitHub

In [1]:
# 📦 Install required packages
!pip install -q numpy pandas matplotlib scikit-image tifffile SimpleITK

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 MB 11.2 MB/s eta 0:00:00


In [ ]:
from skimage.io import imread
import matplotlib.pyplot as plt

In [2]:
# 📂 Download and view a sample brain section
!wget -q -O section_sample.tif https://github.com/AllenInstitute/MapMySections/raw/main/sample_data/section_1234.tif

img = imread("section_sample.tif")
plt.imshow(img, cmap='gray')
plt.title("Sample Brain Section")
plt.axis('off')
plt.show()

TiffFileError: not a TIFF file b''

In [4]:
# 📂 Download and view a sample brain section
!wget -q -O section_sample.tif https://github.com/AllenInstitute/MapMySections/raw/main/sample_data/section_1234.tif

# Check if the file was downloaded successfully and has content
import os
if os.path.exists("section_sample.tif") and os.path.getsize("section_sample.tif") > 0:
    print("File downloaded successfully!")
else:
    print("File download failed or file is empty. Please check your internet connection and try again.")
    exit() # Exit the script if file download failed


img = imread("section_sample.tif")
plt.imshow(img, cmap='gray')
plt.title("Sample Brain Section")
plt.axis('off')
plt.show()

File download failed or file is empty. Please check your internet connection and try again.


TiffFileError: not a TIFF file b''

In [3]:
# 🔁 Synthetic Registration Demo
import SimpleITK as sitk
import numpy as np

# Simulate a fixed image by rotating the sample
fixed_array = np.rot90(img)
fixed_image = sitk.GetImageFromArray(fixed_array.astype(np.float32))
moving_image = sitk.GetImageFromArray(img.astype(np.float32))

# Initialize transform
initial_transform = sitk.CenteredTransformInitializer(
    fixed_image, moving_image, sitk.Euler2DTransform(),
    sitk.CenteredTransformInitializerFilter.GEOMETRY)

# Setup registration
registration_method = sitk.ImageRegistrationMethod()
registration_method.SetMetricAsMattesMutualInformation(50)
registration_method.SetOptimizerAsGradientDescent(learningRate=1.0, numberOfIterations=100)
registration_method.SetInterpolator(sitk.sitkLinear)
registration_method.SetInitialTransform(initial_transform, inPlace=False)

# Run registration
final_transform = registration_method.Execute(fixed_image, moving_image)
resampled_image = sitk.Resample(moving_image, fixed_image, final_transform, sitk.sitkLinear, 0.0)

# Visualize result
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
axes[0].imshow(sitk.GetArrayViewFromImage(fixed_image), cmap='gray')
axes[0].set_title("Fixed")
axes[1].imshow(sitk.GetArrayViewFromImage(moving_image), cmap='gray')
axes[1].set_title("Moving")
axes[2].imshow(sitk.GetArrayViewFromImage(resampled_image), cmap='gray')
axes[2].set_title("Registered")
for ax in axes: ax.axis('off')
plt.tight_layout()
plt.show()

NameError: name 'img' is not defined